# Tutorial #2

This second Hotcent tutorial deals with the optimization of the
electronic DFTB parameters, in particular the adjustment of the
Slater-Koster integrals (done via the confinement potentials).

The general ```hotcent.tools.ConfinementOptimizer``` class handles
such optimizations, if supplied with a residual function and an
initial guess for the confinement parameters (amongst other items).
The residual function is supposed to calculate the deviation
from reference for a given set of (repulsionless) ```*-*.skf```
files (the residual being the metric that needs to be minimized).

Here, we will use the residual function in ```hotcent.tools.DftbPlusBandStructure```
which compares DFTB band structures (using DFTB+) with reference
band structures calculated with e.g. DFT. Continuing with the example
of Si in Tutorial #1, we will consider the band structure of diamond Si.

## Reference data
First, the [GPAW](https://wiki.fysik.dtu.dk/gpaw/) code is used to
generate the reference band structure. This data is handled in the
form of an ```ase.dft.band_structure.BandStructure``` object which
contains e.g. the eigenvalues, Brillouin zone paths, etc. For convenience
this object has already been stored in ```bs_gpaw.json```, hence running
the cell below is optional.

In [ ]:
import numpy as np
from ase.build import bulk
from ase.io.jsonio import write_json
from gpaw import GPAW, PW, Mixer, FermiDirac
from gpaw.eigensolvers import CG


# First perform a regular SCF run
calc = GPAW(mode=PW(400),
            maxiter=200,
            spinpol=False,
            kpts=(5, 5, 5),
            xc='LDA',
            txt='-',
            occupations=FermiDirac(0.02),
            mixer=Mixer(0.05, 8, 100),
            )

atoms = bulk('Si', 'diamond')
atoms.set_calculator(calc)
atoms.get_potential_energy()

# Get the valence band maximum
efermi = calc.get_fermi_level()
Nk = len(calc.get_ibz_k_points())
Ns = calc.get_number_of_spins()
eigval = np.array([[calc.get_eigenvalues(kpt=k, spin=s)
                    for k in range(Nk)] for s in range(Ns)])
evbm = np.max(eigval[eigval < efermi])

# Next, a band structure calculation
calc.set(nbands=8,  # 4 occupied and 4 unoccupied bands
         fixdensity=True,
         eigensolver=CG(niter=5),
         symmetry='off',
         kpts={'path': 'LGXUG', 'npoints': 50},
         convergence={'bands': 'all'},
         )
calc.get_potential_energy()

bs_gpaw = calc.band_structure()
bs_gpaw.reference = evbm
bs_gpaw.plot(filename='bs_gpaw.png', show=False, emax=evbm + 5., emin=evbm - 15.)
write_json('bs_gpaw.json', bs_gpaw)

The resulting band structure plot is shown in ```bs_gpaw.png```:
<img src="./bs_gpaw.png" style="width: 500px;">


## Confinement parameter fitting

Now comes the optimization of the density and wave function confinement potentials for Si,
with the aim of reproducing the above DFT band structure. The total residual will be
calculated with ```hotcent.tools.DftbPlusBandStructure.get_residual``` and corresponds to:<br>

$\mathrm{Residual} = \sum_{\mathrm{bandstructures}\,bs} w_{bs} \sum_{\mathrm{kpoints \in bs}\,k}
                     \sum_{\mathrm{bands}\,n \in k} w_{nk} \left(
                     (\epsilon_{nk}^\mathrm{DFT} - \epsilon_\mathrm{ref}^\mathrm{DFT}) -
                     (\epsilon_{nk}^\mathrm{DFTB} - \epsilon_\mathrm{ref}^\mathrm{DFTB}) \right)^2$,<br>
                     
with the reference energy level $\epsilon_\mathrm{ref}$ usually taken equal to the valence
band maximum ('VBM').<br>
The weights $w_{nk}$ are drawn from the following Boltzmann distribution:<br>
$w_{nk} = \exp \left( \frac{-|\epsilon_{nk}^\mathrm{DFT}
           - \epsilon_\mathrm{ref}^\mathrm{DFT}|}{k_\mathrm{B} T} \right)$,<br>
meaning that more weight will be attached to the eigenvalues closer to the VBM.

For this purpose also a non-minimal basis set will be applied, with inclusion of the
Si 3d states. This yields significantly improved DFTB band structures (you may see for
yourself by omitting these states from the valence).

In [ ]:
# In the following we need the DFTB+ code. The ASE interface needs the executable
# path to be set in the $DFTB_COMMAND environment variable, e.g. like this
# (modify as needed):
%env DFTB_COMMAND=/home/maxime/Downloads/dftbplus-17.1.x86_64-linux/bin/dftb+
# Note that the 17.1 version of DFTB+ is needed here, since later versions
# versions do not include eigenvalues, Fermi levels, ... in the 'results.tag' file,
# which the ASE interface relies upon.

In [ ]:
from ase.io.jsonio import read_json
from ase.units import Bohr
from ase.build import bulk
from ase.data import atomic_numbers, covalent_radii
from hotcent.atomic_dft import AtomicDFT
from hotcent.confinement import PowerConfinement
from hotcent.tools import ConfinementOptimizer, DftbPlusBandStructure


# Setting up the atomic DFT instance(s) and
# calculating the eigenvalues and Hubbard values
atom = AtomicDFT('Si',
                 configuration='[Ne] 3s2 3p2 3d0',
                 valence=['3s', '3p', '3d'],
                 xc='LDA',
                 scalarrel=False,
                 mix=0.05,
                 maxiter=500,
                 confinement=PowerConfinement(r0=40., s=4),
                 txt=None)
atom.run()
atom.info = {}
atom.info['eigenvalues'] = {nl: atom.get_eigenvalue(nl) for nl in atom.valence}

U_p = atom.get_hubbard_value('3p', scheme='central', maxstep=1.)
atom.info['hubbardvalues'] = {'s': U_p}
atom.info['occupations'] = {'3s': 2, '3p': 2, '3d': 0}

# Creating a DFTB+ band structure evaluator and
# supplying it with a reference (DFT) band structure
dpbs = DftbPlusBandStructure(Hamiltonian_SCC='Yes',
                             Hamiltonian_OrbitalResolvedSCC='No',
                             Hamiltonian_MaxAngularMomentum_='',
                             Hamiltonian_MaxAngularMomentum_Si='d',
                             Hamiltonian_PolynomialRepulsive='SetForAll {Yes}')

bs_gpaw = read_json('bs_gpaw.json')  # the reference band structure (DFT)
atoms = bulk('Si', 'diamond')
# see hotcent.tools.DftbPlusBandStructure for more information
# on the various keyword arguments used below
dpbs.add_reference_bandstructure(bs_gpaw, atoms=atoms, kpts_scf=(5, 5, 5),
                                 reference_level='vbm', nsemicore=0, weight=1.,
                                 distribution={'type': 'Boltzmann', 'kBT': 1.5})

# Setting up and running the actual optimizer
# (the keyword arguments are known from hotcent.slako.SlaterKosterTable)
confopt = ConfinementOptimizer(atom, N=500, rmin=0.4, dr=0.02, stride=4,
                               superposition='density', xc='LDA')

# The initial confinement parameters are the same as in Tutorial #1.
# The additional 'adjustable' keyword argument serves to indicate
# which parameters are allowed to vary. Here we keep the quadratic
# form (s=2) and treat the confinement radii r0 as variable.
rcov = covalent_radii[atomic_numbers['Si']] / Bohr
initial_guess = {'Si_3s,Si_3p': PowerConfinement(r0=2 * rcov, s=2, adjustable=['r0']),
                 'Si_3d': PowerConfinement(r0=2 * rcov, s=2, adjustable=['r0']),
                 'Si_n': PowerConfinement(r0=3 * rcov, s=2, adjustable=['r0'])}
# Note that the 'Si_3s,Si_3p' combination indicates that the same
# confinement potential is used for both states. The Si_3d confinement
# (as well as the density confinement) is defined separately and
# their r0 parameters are hence allowed to vary independently.

# Only two iterations are performed here to limit the computational time.
# The confinement therefore hardly changes from the initial guess.
# In real scenarios, typically on the order of 100 iterations are required.
vconf = confopt.run(dpbs.get_residual, initial_guess=initial_guess, tol=1e-2,
                    method='COBYLA', options={'maxiter': 2, 'rhobeg': 0.2})
# See scipy.optimize.minimize for more information on available optimization methods:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

# Make the DFTB band structure plot on the basis of the latest Si-Si.skf file
# (generated with the confinement parameters yielding the lowest residual)
bs_dftb = dpbs.calculate_bandstructure(bs_gpaw)
bs_dftb.plot(filename='bs_dftb.png', emax=bs_dftb.reference + 5, emin=bs_dftb.reference - 15)

In this particular case, the initial guess happens to be fairly good,
and hence the DFTB band structure is quite close to the reference
despite the very limited number of optimization iterations:

DFTB (```bs_dftb.png```):
<img src="./bs_dftb.png" style="width: 300px;">

DFT (```bs_gpaw.png```):
<img src="./bs_gpaw.png" style="width: 300px;">



## Concluding remarks

* Another band structure fitting example (included for testing purposes) can be found in
```examples/bandstructure.py```

* The confinement parameters can also be optimized (in conjunction with the repulsive potentials)
  so as to minimize the residual with respect to relative total energies and forces.
  For this purpose, the ```fit_repulsion``` function in [Tango](https://gitlab.com/mvdb/tango) can
  be used.